# **Generation Similarity**

In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma langchain_groq ragas

In [ ]:
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig
from langchain_core.output_parsers import StrOutputParser
import re
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.storage import InMemoryStore
from operator import itemgetter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.retrievers import EnsembleRetriever # Supports Ensembling of results from multiple retrievers
from langchain_community.retrievers import BM25Retriever
from pydantic import BaseModel, Field
from typing import List
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.colab import userdata
from langchain import PromptTemplate
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import os
import json
from google.colab import files
import time
from langchain_groq import ChatGroq
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import CrossEncoder
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from ragas import SingleTurnSample
from ragas.metrics import ResponseRelevancy, LLMContextRecall
from ragas.llms import LangchainLLMWrapper
import math
import numpy as np

In [ ]:
uploaded_files = files.upload()

In [ ]:
with open('model_response_docs_feed_5_parent_chunk_size_360.json', 'r') as file:
    responses = json.load(file)

In [ ]:
with open('model_results_docs_feed_5_parent_chunk_size_360.json', 'r') as file:
    results = json.load(file)

In [ ]:
with open('generation_eval.json', 'r') as file:
    generation_eval = json.load(file)

cross_encoder_embedings_model_name = "BAAI/bge-reranker-large"
cross_encoder_model = HuggingFaceCrossEncoder(model_name=cross_encoder_embedings_model_name)

def process_response_answer_only(response):
  answer_only = response.split('Citations')[0].replace("\n", " ").strip()
  return answer_only

question_numbers = ['question_1', 'question_2', 'question_3', 'question_4', 'question_5']
similarity_gpt_and_ours = []
for question_number in question_numbers:
  parent_question = generation_eval[question_number]['question']
  ground_truth_answer = generation_eval[question_number]['ground_truth']
  sub_answers = []
  for response in responses:
    for k,v in response.items():
      if response[k]['parent_question'] == parent_question:
        sub_answers.append(process_response_answer_only(response[k]['answer']))
  model_answer = '.'.join(sub_answers)

  similarity_gpt_and_ours.append(cross_encoder_model.score((ground_truth_answer,model_answer)))

In [ ]:
results['similarity_with_ground_truth'] = {
    'all_scores': similarity_gpt_and_ours,
    'mean': np.mean(similarity_gpt_and_ours)
}

In [ ]:
with open('model_results_docs_feed_5_parent_chunk_size_360.json', 'w') as file:
    json.dump(results, file, indent=4)